In [ ]:
!pip install llama-index==0.9.36 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.0 MB/s eta 0:00:00


In [ ]:
import json
from typing import Sequence, List, Union
import nest_asyncio
import tiktoken

from llama_index import  ServiceContext
from llama_index.agent import OpenAIAgent, FnRetrieverOpenAIAgent
from llama_index.llms import  ChatMessage , AzureOpenAI
from llama_index.tools import BaseTool, FunctionTool
from llama_index.callbacks import CallbackManager, TokenCountingHandler

nest_asyncio.apply()

# Token Counting

In [ ]:
token_counter = TokenCountingHandler(
    tokenizer=tiktoken.encoding_for_model("gpt-3.5-turbo").encode
)

callback_manager = CallbackManager([token_counter])

In [ ]:
service_context = ServiceContext.from_defaults(
     callback_manager=callback_manager,llm=None,embed_model=None
)


LLM is explicitly disabled. Using MockLLM.
Embeddings have been explicitly disabled. Using MockEmbedding.


## Tools


In [ ]:
def identification(id: str, birth_date: str) -> bool:
    """Use this tool to preform identification in the system. The function takes in a costumer id as a number and birth date in date_time format (YYYY:MM:DD) and returns true if
    the identification succeeded"""
    return True


identification_tool = FunctionTool.from_defaults(fn=identification)

In [ ]:
def get_costumer_data(id : str) -> str:
    """Use this tool when you need to get costumer meta data. The meta data contains
    the costumer location. The function takes in a costumer id and returns the costumer meta data as a string."""
    return "Tel-aviv"

get_costumer_data_tool = FunctionTool.from_defaults(fn=get_costumer_data)

In [ ]:
def get_costumer_futre_appoinments(id : str) -> list[str]:
    """Use this tool when you need to get costumer future appointments. The function takes in a costumer id and returns all of his future appointments as a list.
    appointment is represented by a tuple of a date and appointment_id."""
    return ["(11.6.23,h123w)"]

get_costumer_futre_appoinments_tool = FunctionTool.from_defaults(fn=get_costumer_futre_appoinments)

In [ ]:
def get_available_profession_appoinments_from_city(city : str, profession: str) -> list[str]:
    """Use this tool when you need to get available appointments for a given profession in a given city."""
    return ["(14.6.23,h456w)","(14.7.23,h362w)"]

get_available_profession_appoinments_from_city_tool = FunctionTool.from_defaults(fn=get_available_profession_appoinments_from_city)

In [ ]:
def schedule_appointment(id : str, appointment_id: str) -> Union[bool , str]:
    """Use this tool when you need to schedule appointment. The function takes a costumer id and an appoinment id and schedule an appointment. If succsessful, return true, else return false and
    other appointment alternatives."""
    return True

schedule_appointment_tool = FunctionTool.from_defaults(fn=schedule_appointment)

### Tool Retriver

In [ ]:
from llama_index import SimpleDirectoryReader, VectorStoreIndex, ServiceContext
from llama_index.embeddings import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="intfloat/e5-base-v2")
service_context = ServiceContext.from_defaults(embed_model=embed_model,llm=None)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

LLM is explicitly disabled. Using MockLLM.


In [ ]:
from llama_index.objects import ObjectIndex, SimpleToolNodeMapping
all_tools = [identification_tool,get_costumer_data_tool,schedule_appointment_tool,get_available_profession_appoinments_from_city_tool
     ,get_costumer_futre_appoinments_tool]


tool_mapping = SimpleToolNodeMapping.from_objects(all_tools)
tools_as_nodes = tool_mapping.to_nodes(all_tools)
tool_index = VectorStoreIndex(nodes=tools_as_nodes,service_context=service_context,top_k=1)
obj_index = ObjectIndex(tool_index,tool_mapping)

In [ ]:
tool_retriver = obj_index.as_retriever(similarity_top_k=2)

In [ ]:
query = "I want to shceudle an appointment"
for i, tool in enumerate(tool_retriver.retrieve(query)):
  print(f'{i+1}. {tool.metadata.name}')

1. schedule_appointment
2. get_available_profession_appoinments_from_city


In [ ]:
def query_tool_pool(query: str):
  """Takes a query and fetch the relevant tool to use"""
  tools = []
  for i, tool in enumerate(tool_retriver.retrieve(query)):
    tools.append(tool.metadata.name)
  return tools


In [ ]:
query_tool_pool("What available appoinments does the costumer have?")

['get_costumer_futre_appoinments', 'get_costumer_data']

In [ ]:
system_prompt = """You are a robotic archivist working with a large archive of texts. You only know how to perform a limited number of operations.
        Your archive contains all the information from the website of a certain bank in the USA.
        The operations are:
        1. Search <terms> [optional list of documents to search in. If omitted, all documents will be searched] - the operation will be performed on the document level,
        so that the function will check for each document if it is relevant to the question (provides information relevant to the question), no aggregation will be performed between the documents.
        The result is a list of documents that are most suitable. You can run subsequent searches in the list of found documents.
        2. Sum <list 1> <list 2> ... <list n> - Sum up the number of elements in all lists.
        3. Minus <list 1> <list 2> - calculates the number of elements in list 1 minus the number of elements in list 2.
        4. Percent <List 1> <List 2> Calculates the number of elements in List 1 divided by the number of elements in List 2.
        5. InRange <list of documents> <Date Time range> - returns the sublist of documents that fall into the range.
        6. last <list of documents> - returns the last document.
        7. first <list. of documents> - returns the first document.
        8. Merge <list of documents> - will turn the list of documents into one document.
        9. Answer <document> <question> - will answer the question based on the content of the document.

        The visitor comes and asks a question.
        Your task is to create a step-by-step plan that will contain only operations from the list of operations provided to you.
        The plan you give should best describe the process that needs to be followed in order to best answer the question, so that it will produce the desired answer in the end."""




In [ ]:
#         Your archive contains Zoom call transcripts of all meetings at a particular company.
system_prompt = """You are a robotic archivist working with a large archive of texts.
        I want you to assume that the information archive is stored using the RAG (Retrieval-Augmented Generation) model so that you can build any subquery you want and the system will return you the k most relevant documents.
        Your archive contains all the information from the website of a certain bank in the USA.

        The visitor comes and asks a question.
        Your task is to create a step-by-step plan to answer the question.
        The plan you give should best describe the process that needs to be followed in order to best answer the question, so that it will produce the desired answer in the end."""

token_counter = TokenCountingHandler(
    tokenizer=tiktoken.encoding_for_model("gpt-3.5-turbo").encode
)

callback_manager = CallbackManager([token_counter])

llm = AzureOpenAI(temperature=0,
                                  engine="vicagpt4-0613",
                                  azure_endpoint="https://vicaopenai.openai.azure.com/",
                                  api_key="8efe6166ff8e47aeab7cbebdf55bf44f",
                                  api_version="2023-07-01-preview",)
agent = OpenAIAgent.from_tools(
    llm=llm, verbose=True,callback_manager=callback_manager, system_prompt = system_prompt) #   tools=all_tools

In [ ]:
agent.reset()

In [ ]:
text = agent.chat("I'm thinking of joining a bank and I want to check the average level of satisfaction between 2015 and 2020.")

Added user message to memory: I'm thinking of joining a bank and I want to check the average level of satisfaction between 2015 and 2020.


In [ ]:
print(text.response)

Step 1: Understand the Query
The first step is to understand the user's query. In this case, the user wants to know the average level of satisfaction of the bank's customers between 2015 and 2020.

Step 2: Formulate Subqueries
Next, we need to formulate subqueries that will help us retrieve the relevant information from the archive. Here are some possible subqueries:
- "Customer satisfaction ratings for the bank from 2015 to 2020"
- "Customer reviews of the bank from 2015 to 2020"
- "Surveys or studies on customer satisfaction of the bank from 2015 to 2020"

Step 3: Retrieve Relevant Documents
Using the RAG model, we will input these subqueries to retrieve the k most relevant documents. The model will use its retrieval component to find the documents that best match the subqueries.

Step 4: Extract Information
Once we have the relevant documents, we will extract the necessary information. This could involve finding average ratings from reviews, or summarizing the findings of surveys or

In [ ]:
print(
    "Embedding Tokens: ",
    token_counter.total_embedding_token_count,
    "\n",
    "LLM Prompt Tokens: ",
    token_counter.prompt_llm_token_count,
    "\n",
    "LLM Completion Tokens: ",
    token_counter.completion_llm_token_count,
    "\n",
    "Total LLM Token Count: ",
    token_counter.total_llm_token_count,
    "\n",
)

Embedding Tokens:  0 
 LLM Prompt Tokens:  913 
 LLM Completion Tokens:  268 
 Total LLM Token Count:  1181 

